<a href="https://colab.research.google.com/github/T2718/AI_Make_Image/blob/main/CivitAI4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# --- モード選択 ---
print("=== モード選択 ===")
print("1: Hugging Faceで画像生成")
print("2: CivitAI（モデルDL / safetensors変換）")
print("3: トークン数を数える")
print("4: 出力フォルダを削除")
mode = input("番号を入力してください: ").strip()

# --- 必要なパラメータを先に全て入力 ---
hf_params = {}
civitai_params = {}

model_list_H = ['stablediffusionapi/eleet-model',
              'stablediffusionapi/brav6',
              'Vsukiyaki/ShiratakiMix',
              'stabilityai/stable-diffusion-xl-base-1.0'
              ]

model_list_C = ["https://civitai.com/api/download/models/28100?type=Model&format=SafeTensor&size=full&fp=fp16",
                "https://civitai.com/api/download/models/11812?type=Model&format=SafeTensor&size=full&fp=fp16"
              ]



if mode == "1":
  hf_params["model_id"] = input("HuggingFaceのモデルIDを入力（例: runwayml/stable-diffusion-v1-5）: ").strip()
  hf_params["submode"] = input("生成モードを選択（1: text2img, 2: img2img）: ").strip()
  hf_params["prompt"] = input("プロンプトを入力: ").strip()
  hf_params["negative_prompt"] = input("ネガティブプロンプトを入力 : ").strip()
  hf_params["num"] = int(input("生成する画像の枚数 : "))
  hf_params["steps"] = int(input("ステップ数（例: 30）: "))
  hf_params["guidance"] = float(input("guidance scale（例: 7.5）: "))
  if hf_params["submode"] == "2":
    hf_params["strength"] = float(input("strength (例: 0.75) : "))
    hf_params["image_path"] = input("初期画像のパスを入力してください（img2img用）: ")

elif mode == "2":
  print("--- CivitAI モデルの状況は？ ---")
  print("1: まだ何もしていない（DLも変換もしていない）")
  print("2: safetensorsファイルのみアップロード済み（変換が必要）")
  print("3: 変換済みでそのまま使える")
  civitai_params["status"] = input("番号を入力してください: ").strip()
  civitai_params["model_name"] = input("保存先モデル名（例: models）: ").strip()
  if civitai_params["status"] == "1":
    civitai_params["url"] = input("CivitAIのモデル/LoRA ダウンロードURL: ").strip()
    civitai_params["need_yaml"] = input("v1-inference.yamlが必要ですか？ (y/n): ").strip().lower() == "y"
  elif civitai_params["status"] == "2":
    civitai_params["safefile"] = input("アップロード済みの .safetensors ファイル名: ").strip()
    civitai_params["need_yaml"] = input("v1-inference.yamlが必要ですか？ (y/n): ").strip().lower() == "y"
  civitai_params["prompt"] = input("プロンプトを入力してください: ")
  civitai_params["negative_prompt"] = input("ネガティブプロンプトを入力 : ").strip()
  civitai_params["num"] = int(input("生成する画像の枚数 : "))
  civitai_params["steps"] = int(input("ステップ数（例: 30）: "))

elif mode == "3":
  token_prompt = input("プロンプトを入力してください（トークン数確認）: ").strip()

elif mode == "4":
  confirm_delete = input("本当にoutputフォルダを削除しますか？ (y/n): ").strip().lower()

# --- 必要なinstallはここで実行（全てのinput終了後） ---
print("インストールを開始します...")
!pip install -q diffusers transformers accelerate safetensors

# --- 各モード実行 ---
if mode == "1":
  from diffusers import StableDiffusionPipeline, StableDiffusionImg2ImgPipeline
  from PIL import Image
  import torch
  import os
  torch_dtype = torch.float16
  if hf_params["submode"] == "1":
    pipe = StableDiffusionPipeline.from_pretrained(
      hf_params["model_id"],
      torch_dtype=torch.float16,
      safety_checker=None  # Safe checkを無効に
    )
    pipe = pipe.to("cuda")  # GPUが必要です
  else:
    pipe = StableDiffusionImg2ImgPipeline.from_pretrained(
        hf_params["model_id"],
        torch_dtype=torch.float16,
        safety_checker=None
    )
    pipe = pipe.to("cuda")
  for k in range(hf_params["num"]):
    if hf_params["submode"] == "1":

      image = pipe(hf_params["prompt"],
                   negative_prompt=hf_params["negative_prompt"],
                   num_inference_steps=hf_params["steps"],
                   guidance_scale=hf_params["guidance"],
                   added_cond_kwargs={}).images[0]
    else:
      init_image = Image.open(hf_params["image_path"]).convert("RGB").resize((512, 512))
      image = pipe(prompt=hf_params["prompt"],
                   negative_prompt=hf_params["negative_prompt"],
                   num_inference_steps=hf_params["steps"],
                   image=init_image,
                   strength=hf_params["strength"],
                   guidance_scale=hf_params["guidance"]).images[0]

    os.makedirs("output", exist_ok=True)
    filename = f"output/huggingface_result"+str(k)+".png"
    image.save(filename)
  print("画像を保存しました")

elif mode == "2":
  import os
  import torch
  import re
  from diffusers import StableDiffusionPipeline

  base_dir = f"./converted/{civitai_params['model_name']}"
  if civitai_params["status"] == "1":
    regex = r'([0-9]{5})'
    if bool(re.match(regex, civitai_params['url'])):
      civitai_params['url'] = 'https://civitai.com/api/download/models/'+civitai_params['url']
    elif civitai_params['url'] == '1':
      civitai_params['url'] = "https://civitai.com/api/download/models/28100?type=Model&format=SafeTensor&size=full&fp=fp16"
    elif civitai_params['url'] == '2':
      civitai_params['url'] = "https://civitai.com/api/download/models/11812?type=Model&format=SafeTensor&size=full&fp=fp16"


    filename = civitai_params["url"].split("/")[-1]
    !wget -q {civitai_params["url"]} -O {filename}
    if civitai_params["need_yaml"]:
      !wget -q https://huggingface.co/runwayml/stable-diffusion-v1-5/resolve/main/v1-inference.yaml
    os.makedirs(base_dir, exist_ok=True)
    !python -m diffusers.pipelines.stable_diffusion.convert_original_stable_diffusion_to_diffusers \
      --checkpoint {filename} \
      --original_config_file v1-inference.yaml \
      --dump_path {base_dir} \
      --use_safetensors

  elif civitai_params["status"] == "2":
    if civitai_params["need_yaml"]:
      !wget -q https://huggingface.co/runwayml/stable-diffusion-v1-5/resolve/main/v1-inference.yaml
    os.makedirs(base_dir, exist_ok=True)
    !python -m diffusers.pipelines.stable_diffusion.convert_original_stable_diffusion_to_diffusers \
      --checkpoint {civitai_params["safefile"]} \
      --original_config_file v1-inference.yaml \
      --dump_path {base_dir} \
      --use_safetensors

  pipe = StableDiffusionPipeline.from_pretrained(base_dir, torch_dtype=torch.float16).to("cuda")
  pipe.enable_xformers_memory_efficient_attention()
  for k in range(civitai_params["num"]):
    image = pipe(prompt=civitai_params["prompt"], negative_prompt=civitai_params["negative_prompt"], num_inference_steps=civitai_params["step"]).images[0]
    os.makedirs("output/"+civitai_params["prompt"][0,10], exist_ok=True)
    image.save("output/"+civitai_params["prompt"][0,10]+"/"+str(k)+".png")
  print("画像を保存しました: output/"+civitai_params["prompt"][0,10]+".png")

elif mode == "3":
  from transformers import CLIPTokenizer
  tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-large-patch14")
  tokens = tokenizer(token_prompt)["input_ids"]
  print(f"トークン数: {len(tokens)}")

elif mode == "4":
  import shutil
  if confirm_delete == "y" and os.path.exists("output"):
    shutil.rmtree("output")
    print("outputフォルダを削除しました")
  else:
    print("キャンセルされました")


=== モード選択 ===
1: Hugging Faceで画像生成
2: CivitAI（モデルDL / safetensors変換）
3: トークン数を数える
4: 出力フォルダを削除
番号を入力してください: 2
--- CivitAI モデルの状況は？ ---
1: まだ何もしていない（DLも変換もしていない）
2: safetensorsファイルのみアップロード済み（変換が必要）
3: 変換済みでそのまま使える
番号を入力してください: 1
保存先モデル名（例: my_model）: models
